# Data Structures & Algorithms

- This episode is more theoretical diving into data structures and algorithms 
- This will allow us to understand what approaches would likely to execute faster than another
- It is important not to obsess over fully understand data structure and algorithms (although it does help, but to recognise slower/faster approaches

## 1.1 Lists

- Lists are a fundamental data structure within Python.
- It is implemented as a form of dynamic array found within many programming languages by different names (C++: std::vector, Java: ArrayList, R: vector, Julia: Vector).
- They allow direct and sequential element access 

How are lists organised?
- Python lists are built on static arrays:
- A static array is a block of contiguous memory (adjacent boxes).
- Each box stores a pointer to a list element.
- Alongside the array, Python tracks the current length (len(my_list)).

Appending when there’s spare space
- The array may be longer than the list so there’s room to grow.
- When you call .append() when there’s space → the item is added to the next empty slot, and the length counter increases.
- If no spare slots remain, Python will allocate a larger array (new contiguous block).
- It will copy all existing pointers into the new array.
- The old array is deallocated.
- The new item is placed at the end.
- Small lists resize often, but as lists grow, resizes become rarer and has a logarithmic growht curve

Conclusion - using append() results in large amount of redundant allocations and copies

What should you use instead? - list comprehensions!
- List comprehension can be twice as fast at building lists than using append()
- This is because list comprehension allows to offload a lot of the computation onto faster C code

- The following benchmark can provide clear results:

In [ ]:
from timeit import timeit

def list_append():
    li = []
    for i in range(100000):
        li.append(i)

def list_preallocate():
    li = [0]*100000
    for i in range(100000):
        li[i] = i

def list_comprehension():
    li = [i for i in range(100000)]

repeats = 1000
print(f"Append: {timeit(list_append, number=repeats):.2f}ms")
print(f"Preallocate: {timeit(list_preallocate, number=repeats):.2f}ms")
print(f"Comprehension: {timeit(list_comprehension, number=repeats):.2f}ms")

- List comprehension was 2x faster, with pre-allocate fairing in the middle
- This also is dependent on python versions, hardware and lists length 

## 1.2 Tuples

- Unlike lists which dynamic arrays, are tuples are immutable static arrays
- That means once created, they cannot be changed and the array cannot be resized 
- They behave more like strings than lists 
- They are useful in representing fixed groups of values 
- Tuples can be joined with a + but a result is always a new tuple (no over allocation)

Why are tuples faster?
- *CPython cahces a lot of short tuples (1-20 elemets)* - EXPLAIN BETTER
- That means creating and destroying them is very cheap and costs little memory 

- We can test the performance of list and tuple allocation using `timeit` module

In [ ]:
python -m timeit "li = [0,1,2,3,4,5]"
python -m timeit "tu = (0,1,2,3,4,5)"

- You can see it takes 3 times longer to allocate a list than tuple the same size

Key takeaway
- Lists: flexible, dynamic, can grow/shrink, but slower to allocate.
- Tuples: fixed, immutable, faster to allocate, useful for grouping stable values.

## 1.3 Dictionaries

- Another fundamental Python data-structure
- Provide key-value store
- There is no intrinsic order on the dictionary: { "a": 1, "b": 2 } is the same dictionary as { "b": 2, "a": 1 }
- They are hashing data structures 

List = one long bookshelf
- When you add a book, it just goes at the end.
- To find a specific book, you might need to scan through every book.
- That’s like searching through a list — slow if the list is big.

Dictionary = bookcase with labeled shelves
- Each book is placed on a shelf based on its genre/author label.
- To find a book, you jump straight to the right shelf.
- That’s like hashing: the key is turned into a label that tells Python exactly where to look.

Why faster?
- With a list, lookup is linear → you may need to check every element.
- With a dictionary, lookup is constant time → go straight to the correct “shelf” using the hash.

(CUSTOM DICTIONARY KEYS: OPTIONAL)

What can be a dictionary key?
- Usually: simple types like numbers or strings.
- Sometimes: you may want to combine values into one compound key.
- Example: a (row, column) tuple to represent a grid cell.
- Even custom objects can be keys — but only if they know how to, for which we need to create a custom class and:
-- Compare equality (__eq__)
-- Produce a hash value (__hash__)

Why hashing matters
- Python uses the hash of the key to decide where in memory to store the value.
- Rule: if two objects are considered equal, they must also have the same hash.
- Otherwise, dictionary lookups would break.

In [ ]:
class MyKey:

    def __init__(self, _a, _b, _c):
        self.a = _a
        self.b = _b
        self.c = _c

    def __eq__(self, other):
        return (isinstance(other, type(self))
                and (self.a, self.b, self.c) == (other.a, other.b, other.c))

    def __hash__(self):
        return hash((self.a, self.b, self.c))

dict = {}
dict[MyKey("one", 2, 3.0)] = 12

## 1.4 Sets

- Dictionary without values - only unique keys
- Equivalent to a mathematical set which has no duplicates, no order
- It is mainly used for elimination of duplicates and for fast checks of element presence

Why are sets fast?
- Use hashing like dictionaries
- That means instead of scanning every element (like a list), Python jumps straight to the right “bucket” to check if an element exists
- This makes in checks and duplicate removal much faster than lists, especially as data grows

In [ ]:
import random
from timeit import timeit

def generateInputs(N = 25000):
    random.seed(12)  # Ensure every list is the same 
    return [random.randint(0,int(N/2)) for i in range(N)]
    
def uniqueSet():
    ls_in = generateInputs()
    set_out = set(ls_in)
    
def uniqueSetAdd():
    ls_in = generateInputs()
    set_out = set()
    for i in ls_in:
        set_out.add(i)
    
def uniqueList():
    ls_in = generateInputs()
    ls_out = []
    for i in ls_in:
        if not i in ls_out:
            ls_out.append(i)

def uniqueListSort():
    ls_in = generateInputs()
    ls_in.sort()
    ls_out = [ls_in[0]]
    for i in ls_in:
        if ls_out[-1] != i:
            ls_out.append(i)
            
repeats = 1000
gen_time = timeit(generateInputs, number=repeats)
print(f"uniqueSet: {timeit(uniqueSet, number=repeats)-gen_time:.2f}ms")
print(f"uniqueSetAdd: {timeit(uniqueSetAdd, number=repeats)-gen_time:.2f}ms")
print(f"uniqueList: {timeit(uniqueList, number=repeats)-gen_time:.2f}ms")
print(f"uniqueListSort: {timeit(uniqueListSort, number=repeats)-gen_time:.2f}ms")

## 1.5 Searching 

When we want to find out if an item exists, the data structure matters a lot for performance.

1. Searching in a set (hashing)

- A set uses hashing to jump straight to the right bucket

- If there are no collisions, the item is found almost immediately. Even with collisions, lookups are usually still very fast.

- On average: O(1) time per search.

2. Searching in a list (linear search)

- Python starts at the first element and checks one by one.

- If the item isn’t there, it will check every single element

- On average, it will search half the list before finding the item.

- On average: O(N) time per search.

3. Searching in a sorted list (binary search)

- If the list is sorted, we can use divide and conquer: 1. look in the middle, decide whether to search the left half or the right half. 2. Repeat until found or not found.

- Each step halves the problem, so it only needs about log₂N checks

- On average: O(log N) time per search.

In [ ]:
import random
from timeit import timeit
from bisect import bisect_left

N = 25000  # Number of elements in list
M = 2  # N*M == Range over which the elements span

def generateInputs():
    random.seed(12)  # Ensure every list is the same
    st = set([random.randint(0, int(N*M)) for i in range(N)])
    ls = list(st)
    ls.sort()  # Sort required for binary
    return st, ls  # Return both set and list
    
def search_set():
    st, _ = generateInputs()
    j = 0
    for i in range(0, int(N*M), M):
        if i in st:
            j += 1
    
def linear_search_list():
    _, ls = generateInputs()
    j = 0
    for i in range(0, int(N*M), M):
        if i in ls:
            j += 1
    
def binary_search_list():
    _, ls = generateInputs()
    j = 0
    for i in range(0, int(N*M), M):
        k = bisect_left(ls, i)
        if k != len(ls) and ls[k] == i:
            j += 1

            
repeats = 1000
gen_time = timeit(generateInputs, number=repeats)
print(f"search_set: {timeit(search_set, number=repeats)-gen_time:.2f}ms")
print(f"linear_search_list: {timeit(linear_search_list, number=repeats)-gen_time:.2f}ms")
print(f"binary_search_list: {timeit(binary_search_list, number=repeats)-gen_time:.2f}ms")